In [1]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import Window

execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
sparkSession = SparkSession.builder.enableHiveSupport().master("local [2]").getOrCreate()

data = sparkSession.read.parquet("/data/sample264")
meta = sparkSession.read.parquet("/data/meta")

In [3]:
window1 = Window.partitionBy('userId').orderBy(col('count').desc())
window2 = Window.partitionBy('userId')

userToTrack = data.groupBy(col("userId"), col("trackId")) \
                  .count() \
                  .withColumn("rank", rank().over(window1)) \
                  .filter(col("rank") <= 1000) \
                  .withColumn('sum_count', sum(col('count')).over(window2)) \
                  .withColumn('norm_count', col('count') / col('sum_count') * 0.5) \
                  .orderBy(desc("norm_count"), asc("userId"), asc("trackId")) \
                  .limit(40) \
                  .select(col("userId"), col("trackId")) \
                  .collect()

In [4]:
for val in userToTrack:
    print "%s %s" % val

66 965774
116 867268
128 852564
131 880170
195 946408
215 860111
235 897176
300 857973
321 915545
328 943482
333 818202
346 864911
356 961308
428 943572
431 902497
445 831381
488 841340
542 815388
617 946395
649 901672
658 937522
662 881433
698 935934
708 952432
746 879259
747 879259
776 946408
784 806468
806 866581
811 948017
837 799685
901 871513
923 879322
934 940714
957 945183
989 878364
999 967768
1006 962774
1049 849484
1057 920458
